<a href="https://colab.research.google.com/github/Himanshu0702/Machine-Learning/blob/main/Tlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import PIL
import PIL.Image as Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
train = '/content/drive/MyDrive/dataset/train'
test = '/content/drive/MyDrive/dataset/test'
validation = '/content/drive/MyDrive/dataset/val'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

    #splits data into training and testing(validation) sets
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/train',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')

test_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/test',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/val',
    target_size=(224, 224),
    batch_size=15,
    class_mode='sparse')

Found 2513 images belonging to 4 classes.
Found 362 images belonging to 4 classes.
Found 717 images belonging to 4 classes.


In [ ]:
feature_extractor = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(224,224,3))

In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_4 (Dropout)         (None, 1280)              0         
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])


In [ ]:
history = model.fit(train_generator, epochs=6, validation_data = validation_generator)

Epoch 1/6
168/168 [==============================] - 122s 713ms/step - loss: 1.1987 - acc: 0.4918 - val_loss: 1.0109 - val_acc: 0.5481
Epoch 2/6
168/168 [==============================] - 116s 693ms/step - loss: 0.9734 - acc: 0.5826 - val_loss: 0.9152 - val_acc: 0.6220
Epoch 3/6
168/168 [==============================] - 115s 687ms/step - loss: 0.9014 - acc: 0.6108 - val_loss: 0.8525 - val_acc: 0.6625
Epoch 4/6
168/168 [==============================] - 117s 697ms/step - loss: 0.8372 - acc: 0.6419 - val_loss: 0.8862 - val_acc: 0.6527
Epoch 5/6
168/168 [==============================] - 115s 682ms/step - loss: 0.7968 - acc: 0.6574 - val_loss: 0.8532 - val_acc: 0.6639
Epoch 6/6
168/168 [==============================] - 115s 682ms/step - loss: 0.7665 - acc: 0.6713 - val_loss: 0.8376 - val_acc: 0.6876


In [ ]:
result = model.evaluate(test_generator)

25/25 [==============================] - 12s 487ms/step - loss: 0.8348 - acc: 0.6547


In [ ]:
img = cv2.imread('/content/drive/MyDrive/dataset/test/Eczema Photos/03EczemaExcoriated011204.jpg')
resize = tf.image.resize(img, (224,224))
yhat = model.predict(np.expand_dims(resize/255,0))
yhat

1/1 [==============================] - 1s 543ms/step


array([[0.11654877, 0.17311364, 0.27753764, 0.43279994]], dtype=float32)

In [ ]:
train_generator.class_indices

{'Atopic Dermatitis Photos': 0,
 'Eczema Photos': 1,
 'Melanoma Skin Cancer Nevi and Moles': 2,
 'Psoriasis pictures Lichen Planus and related diseases': 3}